In [5]:
import pandas as pd
import numpy as np
from data import clients  # our data
import spacy
from pprint import pprint
nlp = spacy.load('en')
from spacy import displacy
import nltk
from terms import RE_MAP,REA_MAP
from pprint import pprint
import re

## load data

In [6]:
df = pd.DataFrame.from_dict(clients)  # load our dirty data in pandas

In [7]:
df

,client_A,client_B,client_C
memo_1,I recently caught up with Adam of Argos Group ...,Retail Property Overview \n * Target in...,I had a call with Carol at Circle Inc to discu...
memo_2,"I will be based in New York, mainly tasked wit...",Industrial Property Overview\n Blue Ind...,They would like to see all core and value-add ...
memo_3,They intend to add a large subterranean retail...,Multi-Family Property Overview\n * Loca...,Circle Inc is aggressively looking to build th...
memo_4,They are seeking large office/residential/reta...,They are open to geographies and can do deals ...,Circle Inc is very focused on the following ma...


## Problem 1
- Multiple iterpretations of a word:
    - multi-house
    - MultiHouse
    - multihouse
    - ...

## Solution!..(?) regular expressions
Why?
Quick and dirty

### Multifamily problem

In [8]:
multi_family = '(?i)mult*\w*\-?fam*\w*'  # mUlTyFamily
#text = df.client_C[3]
text = 'This multifamily is not very good. my Multi-family is better. Not as good as my MULTIFAMILY'
match = re.findall(multi_family,text)
match

['multifamily', 'Multi-family', 'MULTIFAMILY']

## Client A

In [9]:
for memo in df.client_A.index:
    mm = df.client_A[memo]
    match = re.findall(multi_family,mm)
    print('{}:{}'.format(memo,match))

memo_1:[]
memo_2:[]
memo_3:[]
memo_4:[]


## Client B

In [10]:
for memo in df.client_B.index:
    mm = df.client_B[memo]
    match = re.findall(multi_family,mm)
    print('{}:{}'.format(memo,match))

memo_1:[]
memo_2:['multifamily']
memo_3:['Multi-Family']
memo_4:[]


## Client C

In [11]:
for memo in df.client_C.index:
    mm = df.client_C[memo]
    match = re.findall(multi_family,mm)
    print('{}:{}'.format(memo,match))

memo_1:[]
memo_2:['multifamily']
memo_3:[]
memo_4:['multi-family', 'Multi-family', 'multi-family', 'multi-family']


## find money values in text

In [13]:
money = '\$?(?i)\d{1,3}[m|b]\w*'  # CREAM
#text = df.client_C[3]
text = '''
But you ain't a Saint, this ain't KumbaYe
But you got hurt because you did cool by 'Ye
You gave him 20mm without blinkin'
He gave you 20 minutes on stage,...
'''
match = re.findall(money,text)
match

['20mm']

# Client A

In [14]:
for memo in df.client_A.index:
    mm = df.client_A[memo]
    match = re.findall(money,mm)
    print('{}:{}'.format(memo,match))

memo_1:[]
memo_2:['$30M']
memo_3:[]
memo_4:['70mm']


## client B
- cant find `$5.5B`

In [15]:
for memo in df.client_B.index:
    mm = df.client_B[memo]
    match = re.findall(money,mm)
    print('{}:{}'.format(memo,match))

memo_1:[]
memo_2:[]
memo_3:['5B', '$250M', '$250M', '$100M']
memo_4:['$50mm']


## Client C

In [16]:
for memo in df.client_C.index:
    mm = df.client_C[memo]
    match = re.findall(money,mm)
    print('{}:{}'.format(memo,match))

memo_1:[]
memo_2:[]
memo_3:[]
memo_4:[]


## Problem 2
- domain specific words create problems with NER
- ROC,HOA,..

## Solution
- replace words from domain dictionary

In [17]:
# function to display domain acronims
def domain_words(text):
    for word in text.split():
        if word  in RE_MAP:
            print('{}: {}'.format(word,RE_MAP[word]))
            #text = text.replace(word, RE_MAP[word])
        if word  in REA_MAP:
            print('{}: {}'.format(word,REA_MAP[word]))
            #text = text.replace(word, REA_MAP[word])
    #return text

## Client A

In [19]:
for memo in df.client_A.index:
    mm = df.client_A[memo]
    dd = domain_words(mm)
    print(dd)

None
JV: Joint Venture
M&A:  Mergers and Acquisitions
None
None
None


## Client B

In [20]:
for memo in df.client_B.index:
    mm = df.client_B[memo]
    dd = domain_words(mm)
    print(dd)

None
REIT: Real Estate Investment Trust
REIT: Real Estate Investment Trust
IRR: Internal Rate of Return
None
None
None


## Client C

In [21]:
for memo in df.client_C.index:
    mm = df.client_C[memo]
    dd = domain_words(mm)
    print(dd)

None
None
MH: Manufactured Housing
None
None
